In [ ]:
import sqlalchemy
import pandas as pd
import numpy as np
from polygon import RESTClient
import datetime as dt
import os
from dotenv import load_dotenv
from connect import engine, Base, Stock_Splits
from log_config import setup_logging
from sqlalchemy import select, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import logging

In [ ]:
setup_logging()

In [ ]:
load_dotenv()

In [ ]:
key = os.getenv("API_KEY")

In [ ]:
client = RESTClient(api_key=key)

In [ ]:
wiki = 'http://en.wikipedia.org/wiki'
djia_ticker_list = wiki + '/Dow_Jones_Industrial_Average'
sp500_tickers_list = wiki + '/List_of_S%26P_500_companies'
tickersSP500 = pd.read_html(sp500_tickers_list)[0].Symbol.to_list()
djia_tickers = pd.read_html(djia_ticker_list)[1].Symbol.to_list()

In [ ]:
class Stock_Splits_update:
    " Class to update stock split data from Polygon.io"

    def __init__ (self,tickers, engine, limit = 1000):
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.engine = engine
        self.limit = limit
        
    def transform_data(self,df,ticker):
        df['ticker '] = ticker
        df['execution_date'] = pd.to_datetime(df['execution_date'])
        return df.to_dict(orient = 'records')
    
    def update_data(self, client):
        with self.engine.connect() as conn:
            for ticker in self.tickers:
                splits = pd.DataFrame(client.list_splits(ticker, limit = self.limit))
                if not splits.empty:
                    transformed_data = self.transform_data(splits, ticker)
                    try:    
                        conn.execute(Stock_Splits.__table__.insert(),transformed_data)
                        conn.commit()
                            
                    except Exception as e:
                        logging.error(f"Error updating stock splits for {ticker}: {e}")
                        continue

In [ ]:
updater = Stock_Splits_update(engine=engine, tickers= 'AAPL')

In [ ]:
updater.update_data(client)